<a href="https://colab.research.google.com/github/maitri2905/Matrix_Factorization/blob/main/Copy_of_Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()

Saving input1.csv to input1.csv


In [35]:
import numpy as np
import pandas as pd
import csv
import io

class Matrix_Factorization():

    def __init__(self, input_matrix_df, dimension, learning_rate, regularization_param, iterations):
      
        self.input_matrix_df = input_matrix_df
        self.users, self.items = input_matrix_df.shape
        self.dimension = dimension
        self.learning_rate = learning_rate
        self.regularization_param = regularization_param
        self.iterations = iterations

    def train(self):

        self.randon_latent_feature_matrices_users = np.random.normal(scale=1./self.dimension, size=(self.users, self.dimension))
        self.randon_latent_feature_matrices_items = np.random.normal(scale=1./self.dimension, size=(self.items, self.dimension))

        self.bias_users = np.zeros(self.users)
        self.bias_items = np.zeros(self.items)
        self.biases = np.mean(self.input_matrix_df[np.where(self.input_matrix_df != 0)])
import numpy as np
import pandas as pd
import csv
import io

class Matrix_Factorization():

    def __init__(self, input_matrix_df, dimension, learning_rate, regularization_param, iterations):
      
        self.input_matrix_df = input_matrix_df
        self.users, self.items = input_matrix_df.shape
        self.dimension = dimension
        self.learning_rate = learning_rate
        self.regularization_param = regularization_param
        self.iterations = iterations

    def train(self):

        self.randon_latent_feature_matrices_users = np.random.normal(scale=1./self.dimension, size=(self.users, self.dimension))
        self.randon_latent_feature_matrices_items = np.random.normal(scale=1./self.dimension, size=(self.items, self.dimension))

        self.bias_users = np.zeros(self.users)
        self.bias_items = np.zeros(self.items)
        self.biases = np.mean(self.input_matrix_df[np.where(self.input_matrix_df != 0)])

        self.training_sample_list = [
            (user, item, self.input_matrix_df[user, item])
            for user in range(self.users)
            for item in range(self.items)
            if self.input_matrix_df[user, item] > 0
        ]

        training = []
        for iteration in range(self.iterations):
            np.random.shuffle(self.training_sample_list)
            self.sgd_step()
            mean_sqare_error = self.mean_sqare_error()
            training.append((iteration, mean_sqare_error))
            if (iteration+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (iteration+1, mean_sqare_error))

        return training

    def mean_sqare_error(self):
       
        users, items = self.input_matrix_df.nonzero()
        prediction = self.full_matrix()
        error = 0
        for user, item in zip(users, items):
            error += pow(self.input_matrix_df[user, item] - prediction[user, item], 2)
        return np.sqrt(error)
        

    def rating(self, i, j):
       
        prediction = self.biases + self.bias_users[i] + self.bias_items[j] + self.randon_latent_feature_matrices_users[i, :].dot(self.randon_latent_feature_matrices_items[j, :].T)
        return prediction

    def sgd_step(self):
       
        for user, item, value in self.training_sample_list:
            prediction = self.rating(user, item)
            error = (value - prediction)

            self.bias_users[user] += self.learning_rate * (error - self.regularization_param * self.bias_users[user])
            self.bias_items[item] += self.learning_rate * (error - self.regularization_param * self.bias_items[item])

            self.randon_latent_feature_matrices_users[user, :] += self.learning_rate * (error * self.randon_latent_feature_matrices_items[item, :] - self.regularization_param * self.randon_latent_feature_matrices_users[user,:])
            self.randon_latent_feature_matrices_items[item, :] += self.learning_rate * (error * self.randon_latent_feature_matrices_users[user, :] - self.regularization_param * self.randon_latent_feature_matrices_items[item,:])

   

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.biases + self.bias_users[:,np.newaxis] + self.bias_items[np.newaxis:,] + self.randon_latent_feature_matrices_users.dot(self.randon_latent_feature_matrices_items.T)


input_matrix_df = pd.read_csv(io.BytesIO(uploaded['input1.csv']))

input_matrix_df = np.array(input_matrix_df)
 
matrix_obj = Matrix_Factorization(input_matrix_df, dimension=3, learning_rate=0.0001, regularization_param= 0.01, iterations=20)
train_output = matrix_obj.train()

Iteration: 10 ; error = 62.5974
Iteration: 20 ; error = 62.4485


In [20]:
print(obj.full_matrix())
print()


[[ 3.68334717  8.22365911 -0.03519204  5.01719678  7.84023115  5.22056796
   2.58950837  1.64706467  3.39809582  6.78843144  6.59739721  4.81628451
   3.39870134  5.71642069  5.13864608  7.48023066  3.97013892  6.27912869
   6.00307651  4.67768633  3.10730092  5.77246742  4.87924947  2.98705792
   3.50773602  4.07154169  3.35996182  4.54741395  4.21693274  0.93945802
   4.97573246  2.59818608  1.42811781  5.38402169  7.01442914  3.81577255
   6.08468813  6.03050225  4.09720876  5.01264266  7.14851408  5.51364383
   4.28877207]
 [ 3.61891555  5.34828811  0.49564514  5.1926585   7.44952399  3.92383997
   5.95021711  5.80641433  1.97698233  5.70299677  3.27900769  8.57839773
   1.9641193   5.75999728  4.27615285  4.76702748  4.56079406  2.39496183
   0.67392091  7.61876803  5.22219699  5.82681555  6.75470117  4.32350207
   5.98724977  6.74511639  1.22311239  6.14989581  4.68665169  2.30896894
   6.41995637  1.65856649  4.60251341  3.57063428  6.1875463   3.55728332
   4.09926782  8.534743